In [2]:
import os
import glob
import numpy as np
import pprint
import functools

import tensorflow as tf

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

In [3]:
import sys
sys.executable

'/home/lcances/.miniconda3/envs/tensorboard/bin/python'

In [4]:
# global variables
tag_list = []
log_list = []
file_stats = dict()
stat_list = ["mean", "std", "maxi", "mini"]

os.chdir("../..")
tensorboard_root = "tensorboard"

# Utility function

In [5]:
def get_all_tag(path):
    global tag_list
    tag_list = []

    for e in tf.compat.v1.train.summary_iterator(path):
        for value in e.summary.value:
            if value.HasField("simple_value"):
                if value.tag not in tag_list:
                    tag_list.append(value.tag)
    
    return tag_list

- Si la list n'est pas vide
    - Si le premier element est un fichier: return True
    - Si le premier element est un dossier:
        - lister le contenue de ce dossier L
        - si L n'est pas vide
            - Si le premier element est un fichier: return True
            - Sinon return False

In [6]:
def contain_log(path):
    if path:
        if os.path.isfile(path[0]):
            return True
        
        if os.path.isdir(path[0]):
            if not os.listdir(path[0]): # directory is empty
                return False
            
            sublist = [os.path.join(path[0], p) for p in os.listdir(path[0])]
            
            if sublist:
                if os.path.isfile(sublist[0]):
                    return True
                else:
                    return False
    return True

In [7]:
def get_tag_list(log_file):
    list_file = os.listdir(log_file)
    
    print(log_file)
    print(list_file)
    path = os.path.join(log_file, list_file[0])
    tag_list = get_all_tag(path)
    print(tag_list)

In [8]:
tensorboard_root

'tensorboard'

In [9]:
!pwd

/home/lcances/sync/Documents_sync/Projet/Deep-Co-Training


In [10]:
glob.glob(tensorboard_root + "/**")

['tensorboard/ubs8k']

In [11]:
def ui_dropdown_dir(directory):
    global log_list
    
    sub_directories = sorted([os.path.join(directory, p) for p in os.listdir(directory) ])
        
    if not contain_log(sub_directories):
        interact(ui_dropdown_dir, directory=sub_directories)
        
    else:
        path_list = [os.path.join(directory, l) for l in os.listdir(directory)]
        log_list = path_list
        interact_manual(get_tag_list, log_file=path_list)

In [12]:
interact(ui_dropdown_dir, directory=glob.glob(tensorboard_root + "/**"))

interactive(children=(Dropdown(description='directory', options=('tensorboard/ubs8k',), value='tensorboard/ubs…

<function __main__.ui_dropdown_dir(directory)>

In [13]:
!ls ../../tensorboard/ubs8k

ls: cannot access '../../tensorboard/ubs8k': No such file or directory


In [14]:
def stat_stat(stat = "maxi"):
    global file_stats
    
    values = [file_stats[l][stat] for l in file_stats.keys()]
    
    print("stat stat")
    print("mean: ", np.nanmean(values))
    print("std : ", np.nanstd(values))
    print("mini: ", np.nanmin(values))
    print("maxi: ", np.nanmax(values))
    print("")
    print("%.3f ± %.3f" % (np.nanmean(values), np.nanstd(values)))
    print("")
    print("detail ")
    pprint.pprint(values)

In [15]:
glob.glob(tensorboard_root + "/**")

['tensorboard/ubs8k']

In [16]:
def ui_dropdown_dir(directory):
    global log_list
    
    sub_directories = sorted([os.path.join(directory, p) for p in os.listdir(directory) ])
        
    if not contain_log(sub_directories):
        interact(ui_dropdown_dir, directory=sub_directories)
        
    else:
        path_list = [os.path.join(directory, l) for l in os.listdir(directory)]
        log_list = path_list
        interact_manual(get_tag_list, log_file=path_list)

In [17]:
interact(ui_dropdown_dir, directory=glob.glob(tensorboard_root + "/**"))

interactive(children=(Dropdown(description='directory', options=('tensorboard/ubs8k',), value='tensorboard/ubs…

<function __main__.ui_dropdown_dir(directory)>

In [18]:
!ls ../../tensorboard/ubs8k

ls: cannot access '../../tensorboard/ubs8k': No such file or directory


In [19]:
def stat_stat(stat = "maxi"):
    global file_stats
    
    values = [file_stats[l][stat] for l in file_stats.keys()]
    
    print("stat stat")
    print("mean: ", np.nanmean(values))
    print("std : ", np.nanstd(values))
    print("mini: ", np.nanmin(values))
    print("maxi: ", np.nanmax(values))
    print("")
    print("%.3f ± %.3f" % (np.nanmean(values) * 100, np.nanstd(values) * 100))
    print("")
    print("detail ")
    pprint.pprint(values)

In [23]:
def statistics(mean=True, std=True, max=False, min=False, tag=tag_list[0]):
    global file_stats
    
    file_log = dict()
    
    # read all the log files ------------------------------
    @functools.lru_cache()
    def get_log_data(path):
        logs = dict()

        for e in tf.compat.v1.train.summary_iterator(path):
            for value in e.summary.value:
                if value.HasField("simple_value"):
                    if value.tag not in logs:
                        logs[value.tag] = []
                    logs[value.tag].append(value.simple_value)
    
        return logs
    
    for l in log_list:
        final_path = os.path.join(l, os.listdir(l)[0])
        logs = get_log_data(final_path)
        
        if logs:
            file_log[l] = logs
        
    # compute the statistics ------------------------------
    # ---- file wise ----
    file_stats = dict()
    
    for l in file_log.keys():
        print("nb sample: ", len(file_log[l][tag]))
        file_stats[l] = dict(
            mean = np.nanmean(file_log[l][tag]),
            std = np.nanstd(file_log[l][tag]),
            mini = np.nanmin(file_log[l][tag]),
            maxi = np.nanmax(file_log[l][tag]),
        )
        
    interact(stat_stat, stat=stat_list)

In [25]:
interact(ui_dropdown_dir, directory=glob.glob(tensorboard_root + "/**"))

interactive(children=(Dropdown(description='directory', options=('tensorboard/ubs8k',), value='tensorboard/ubs…

<function __main__.ui_dropdown_dir(directory)>

In [26]:
interact_manual(statistics, mean=True, std=True, mini=False, maxi=False, tag=tag_list)

interactive(children=(Checkbox(value=True, description='mean'), Checkbox(value=True, description='std'), Check…

<function __main__.statistics(mean=True, std=True, max=False, min=False, tag='train/Lce')>

In [37]:
def contain_log(path):
    if path:
        
        if os.path.isfile(path[0]):
            return True
        
        if os.path.isdir(path[0]):
            sublist = [os.path.join(path[0], p) for p in os.listdir(path[0])]
            
            if sublist:
                if os.path.isfile(sublist[0]):
                    return True
                else:
                    return False
    return True

In [38]:
root = "/mnt/e/sync/Documents_sync/Projet/UrbanSound8K/tensorboard/osirim_tensorboard"
list_dir = os.listdir(root)

for name in list_dir:
    path = os.path.join(root, name)
    
    print(contain_log(path))

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/e/sync/Documents_sync/Projet/UrbanSound8K/tensorboard/osirim_tensorboard'

In [14]:
interact(ui_dropdown_dir, directory=glob.glob(tensorboard_root + "/**"))

interactive(children=(Dropdown(description='directory', options=('../tensorboard/olympe_tensorboard', '../tens…

<function __main__.ui_dropdown_dir(directory)>